In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

conf = (SparkConf().setMaster("k8s://https://192.168.219.100:6443") # Your master address name
        .set("spark.kubernetes.container.image", "joron1827/pyspark:v2") # Spark image name
        .set("spark.driver.port", "2222") # Needs to match svc
        .set("spark.driver.blockManager.port", "7777")
        .set("spark.driver.host", "driver-service.jupyterhub.svc.cluster.local") # Needs to match svc
        .set("spark.driver.bindAddress", "0.0.0.0")
        .set("spark.kubernetes.namespace", "spark")
        .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
        .set("spark.kubernetes.authenticate.serviceAccountName", "spark")
        .set("spark.executor.instances", "4")
        .set("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")
        .set("spark.app.name", "joronSpark")
        .set("spark.executor.cores", "4")
        .set("spark.executor.memory", "16g"))

In [2]:

spark = SparkSession.builder.config(conf=conf).getOrCreate()

import time

time.sleep(10)

/home/jovyan/.local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/06/02 04:41:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.parquet("hdfs://192.168.219.121:9000/crawling_stock_text_merge/two_class_text_vectorized.parquet")
df.show()

+------+----------+-------------------------------+----------+-----------------------------+--------------------+--------------------+--------------------+
|  code|      date|                          title|New_Column|                       tokens|                  tf|               tfidf|            word2vec|
+------+----------+-------------------------------+----------+-----------------------------+--------------------+--------------------+--------------------+
|035720|2017-09-26|                               |         1|                           []|      (262144,[],[])|      (262144,[],[])|         (100,[],[])|
|003550|2022-03-16|                      주주에...|         0|   [주주, 원한, 잇는건, 가요]|(262144,[51217,78...|(262144,[51217,78...|[0.01652339892461...|
|323410|2021-08-09|                           이이|         1|                       [이이]|(262144,[104757],...|(262144,[104757],...|[0.03197551518678...|
|007700|2020-06-26|                        와 소름|         0|                     

In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 학습 데이터와 테스트 데이터 분할 (70% 학습, 30% 테스트)
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

# 로지스틱 회귀 모델 초기화
lr = LogisticRegression(labelCol="New_Column", featuresCol="tfidf")

# 모델 학습
model = lr.fit(train_data)

# 테스트 데이터에 대한 예측
predictions = model.transform(test_data)

# 예측 결과 확인
predictions.select("New_Column", "prediction", "probability").show()

# 이진 분류 평가 지표 계산 (예시로는 Area Under ROC를 사용)
evaluator = BinaryClassificationEvaluator(labelCol="New_Column")
auc = evaluator.evaluate(predictions)

predictions.show()
print("Area Under ROC:", auc)


23/06/02 04:42:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/02 04:42:09 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/06/02 04:42:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/06/02 04:42:09 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


23/06/02 04:42:38 WARN DAGScheduler: Broadcasting large task binary with size 1248.8 KiB


+----------+----------+--------------------+
|New_Column|prediction|         probability|
+----------+----------+--------------------+
|         1|       1.0|[0.13685368423321...|
|         1|       1.0|[0.24140023628268...|
|         1|       1.0|[0.38090712684037...|
|         1|       1.0|[0.16827606561352...|
|         1|       1.0|[0.00114636375802...|
|         1|       1.0|[0.26074411303739...|
|         1|       1.0|[0.38434862811889...|
|         0|       1.0|[1.77485957549464...|
|         0|       0.0|[0.68700323968091...|
|         1|       1.0|[0.16591234548293...|
|         1|       0.0|[0.99999995568491...|
|         0|       1.0|[0.49689513079298...|
|         0|       1.0|[0.18217071543739...|
|         0|       1.0|[0.27674792946181...|
|         0|       0.0|[0.65777770066362...|
|         0|       1.0|[0.30623734010214...|
|         0|       0.0|[0.56322801549791...|
|         0|       1.0|[0.11961904174933...|
|         1|       1.0|[0.19011011870991...|
|         

23/06/02 04:42:43 WARN DAGScheduler: Broadcasting large task binary with size 1264.9 KiB


+------+----------+------------------------------------+----------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  code|      date|                               title|New_Column|                        tokens|                  tf|               tfidf|            word2vec|       rawPrediction|         probability|prediction|
+------+----------+------------------------------------+----------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|000020|2020-06-12|   요기 고수가 어딧냐 ㄱ 초보들뿐...|         1|  [요기, 고수, 어딧, 냐, ㄱ...|(262144,[43344,77...|(262144,[43344,77...|[-0.0527202452596...|[-1.8416718630815...|[0.13685368423321...|       1.0|
|000020|2020-07-29|                   동화약품과 제테마|         1|              [동화약품, 테마]|(262144,[156516,1...|(262144,[156516,1...|[0.15325093083083...|[-1.1450180303

In [5]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# RandomForestClassifier 모델을 생성합니다
rf = RandomForestClassifier(labelCol="New_Column", featuresCol="tfidf")

# 파이프라인을 생성합니다
pipeline = Pipeline(stages=[rf])

# 모델을 학습시킵니다
model = pipeline.fit(train_data)  # trainData는 학습에 사용할 데이터 프레임입니다

# 학습된 모델을 사용하여 예측을 수행합니다
predictions = model.transform(test_data)  # testData는 예측에 사용할 데이터 프레임입니다

# 예측 결과 확인
predictions.select("New_Column", "prediction", "probability").show()

23/06/02 04:43:38 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/06/02 04:43:51 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB


23/06/02 04:43:51 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


ERROR:root:KeyboardInterrupt while sending command.               (0 + 16) / 17]
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/jovyan/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# MulticlassClassificationEvaluator를 사용하여 정확도를 계산합니다
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)


Accuracy: 0.6551456158935797


In [6]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# MulticlassClassificationEvaluator를 사용하여 정밀도, 재현율, F1 스코어를 계산합니다
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction")

# 정밀도 계산
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})

# 재현율 계산
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

# F1 스코어 계산
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


23/06/02 04:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1257.8 KiB


ERROR:root:KeyboardInterrupt while sending command.>               (0 + 0) / 17]
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/jovyan/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 의사결정트리 모델 생성
dt = DecisionTreeClassifier(labelCol="New_Column", featuresCol="tfidf")

# 모델 훈련
dt_model = dt.fit(train_data)

# 테스트 데이터에 대한 예측
dt_predictions = dt_model.transform(test_data)

# 평가 지표 계산을 위한 MulticlassClassificationEvaluator 생성
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction")

# 정확도 계산
accuracy = evaluator.evaluate(dt_predictions, {evaluator.metricName: "accuracy"})

# 정밀도 계산
precision = evaluator.evaluate(dt_predictions, {evaluator.metricName: "weightedPrecision"})

# 재현율 계산
recall = evaluator.evaluate(dt_predictions, {evaluator.metricName: "weightedRecall"})

# F1 스코어 계산
f1_score = evaluator.evaluate(dt_predictions, {evaluator.metricName: "f1"})

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


23/06/02 04:47:15 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


23/06/02 04:47:53 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/06/02 04:48:18 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB


23/06/02 04:48:18 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


23/06/02 04:51:12 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


...

23/06/02 04:55:04 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


23/06/02 04:56:55 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_555_4 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_429_6 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_555_6 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_429_2 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_429_14 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_429_10 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_555_2 !
23/06/02 04:57:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_555_0 !
23/06/02 04:57:39 ERROR TaskSchedulerImpl: Lost executor 3 on 10.244.3.179: 
The executor with id 3 exited with exit code 137(SIGKILL, possible container OOM).



The API gave the following container statuses:


	 container name: spark-kubernetes

23/06/02 05:00:32 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


23/06/02 05:03:18 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


In [7]:
spark.stop()

23/06/02 03:56:45 ERROR Instrumentation: org.apache.spark.SparkException: Job 146 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1188)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1186)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1186)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2887)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2784)
	at org.apache.spark.SparkContext.$anonfun$stop$11(SparkContext.scala:2095)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1484)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:2095)
	at org.apache.spark.api.java.JavaSparkContext.stop(JavaS